# Aula 10 - Computational Tradeoffs

[Unicamp - IA368DD: Deep Learning aplicado a sistemas de busca.](https://www.cpg.feec.unicamp.br/cpg/lista/caderno_horario_show.php?id=1779)

Autor: Marcus Vinícius Borela de Castro

[Repositório no github](https://github.com/marcusborela/deep_learning_em_buscas_unicamp)


# Enunciado exercício



Exercício desta semana: Trade-offs de eficiência e qualidade

O objetivo do exercício desta semana é construir alguns pipelines de busca e analisá-los em termos das seguintes métricas:
Qualidade dos resultados: nDCG@10;
Latência (seg/query);
USD por query assumindo utilização "perfeita": assim que terminou de processar uma query, já tem outra para ser processada;
USD/mês para deixar o sistema rodando para poucos usuários (ex: 100 queries/dia);
Custo de indexação em USD;

Iremos avaliar os pipelines no TREC-COVID.
A latência precisa ser menor que 2 segundos por query.
Não assumir processamento de queries em batch.

Considerar:
1,50 USD/hora por A100 ou 0,21 USD/hora por T4 ou 0,50 USD/hora por V100
0,03 USD/hora por CPU core
0,005 USD/hora por GB de CPU RAM
Dicas:
Utilizar modelos de busca "SOTA" já treinados no MS MARCO como parte do pipeline, como o SPLADE distil (esparso), contriever (denso), Colbert-v2 (denso), miniLM (reranker), monoT5-3B (reranker), doc2query minus-minus (expansão de documentos + filtragem com reranqueador na etapa de indexação)
Pode usar API's como Cohere, OpenAI Embeddings

Variar parâmetros como número de documentos retornados em cada estágio. Por exemplo, BM25 retorna 1000 documentos, um modelo denso ou esparso pode franqueá-los, e passar os top 50 para o miniLM/monoT5 fazer um ranqueamento final.



# Organizando o ambiente

## Importações

In [1]:
import pandas as pd

In [2]:
from tqdm import tqdm

In [3]:
import os

In [4]:
import pickle

In [5]:
import time

In [6]:
import numpy as np

In [7]:
os.environ['JVM_PATH'] = '/usr/lib/jvm/java-11-openjdk-amd64/lib/server/libjvm.so'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'

In [8]:
from pyserini.search.lucene import LuceneSearcher

/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from evaluate import load

In [10]:
import torch

In [11]:
import gzip, json

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [13]:
from torch.utils.data import DataLoader

In [14]:
from torch.utils.data import Dataset

In [15]:
from transformers import BatchEncoding

## Definindo paths

In [16]:
DIRETORIO_LOCAL = '/home/borela/fontes/deep_learning_em_buscas_unicamp/local'
DIRETORIO_TREC_COVID = F'{DIRETORIO_LOCAL}/trec_covid'
DIRETORIO_MSMARCO = F'{DIRETORIO_LOCAL}/msmarco'
DIRETORIO_TRABALHO = F'{DIRETORIO_LOCAL}/tradeoff'
DIRETORIO_RUN = f"{DIRETORIO_TRABALHO}/runs"
PATH_RUN_AVALIACAO = f"{DIRETORIO_RUN}/run-trec-covid.txt"
PATH_RESULTADO_PIPELINE = f"{DIRETORIO_TRABALHO}/resultado_pipeline.pickle"
PATH_AVALIACAO_CONTEXTO_PIPELINE = f"{DIRETORIO_TRABALHO}/avaliacao_contexto_pipeline.pickle"

In [17]:
if os.path.exists(DIRETORIO_LOCAL):
    print('pasta já existia!')
else:
    os.makedirs(DIRETORIO_LOCAL)
    print('pasta criada!')


pasta já existia!


In [18]:
if os.path.exists(DIRETORIO_TRABALHO):
    print('pasta já existia!')
else:
    os.makedirs(DIRETORIO_TRABALHO)
    print('pasta criada!')


pasta já existia!


In [19]:
if os.path.exists(DIRETORIO_RUN):
    print('pasta já existia!')
else:
    os.makedirs(DIRETORIO_RUN)
    print('pasta criada!')


pasta já existia!


## Outras inicializações

In [20]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Construindo o índice prebuilt trec-covid

In [21]:
tempo_inicio = time.time()
LuceneSearcher.from_prebuilt_index('beir-v1.0.0-trec-covid.flat')
tempo_construcao_indice_trecc_pyserine = round(time.time() - tempo_inicio, 6)

In [22]:
print(f"Tempo para construir índice TRECC com pyserini {tempo_construcao_indice_trecc_pyserine} segundos")

Tempo para construir índice TRECC com pyserini 0.109634 segundos


In [23]:
os.getcwd()

'/home/borela/fontes/deep_learning_em_buscas_unicamp/code/aula_10_computational_tradeoffs'

In [24]:
!ls /home/borela/.cache/pyserini/indexes/

lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade


In [25]:
NOME_INDICE_TRECC = os.popen('ls /home/borela/.cache/pyserini/indexes/').read()[:-1]
PATH_INDICE_TRECC = "/home/borela/.cache/pyserini/indexes/" + NOME_INDICE_TRECC

In [26]:
!ls  -l --block-size=1 /home/borela/.cache/pyserini/indexes/

total 4096
drwxrwxr-x 2 borela borela 4096 nov 16 12:21 lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade


In [27]:
print(NOME_INDICE_TRECC)

lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade


In [28]:
tamanho_em_bytes_indice_trecc = os.path.getsize(PATH_INDICE_TRECC)
print("O tamanho do arquivo", NOME_INDICE_TRECC, "é:", tamanho_em_bytes_indice_trecc, "bytes")

O tamanho do arquivo lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade é: 4096 bytes


O tamanho não está correto. Mas continuaremos o código deixando esse ponto pendente

In [29]:
from pyserini.index import IndexReader


In [30]:
index_reader = IndexReader(PATH_INDICE_TRECC)
num_docs = index_reader.stats()
print(f'Número de documentos no índice: {num_docs}')

Número de documentos no índice: {'total_terms': 20822821, 'documents': 171331, 'non_empty_documents': 171331, 'unique_terms': 202648}


In [31]:
os.path.exists(PATH_INDICE_TRECC)

True

In [32]:
import subprocess


In [33]:

tamanho_em_bytes_indice_trecc = int(subprocess.check_output(['du', '-b', PATH_INDICE_TRECC]).split()[0])


In [34]:
tamanho_em_bytes_indice_trecc

269772727

# Baixando os dados e preparando para avaliação 

## Documentos

In [35]:
if not os.path.exists(f"{DIRETORIO_TREC_COVID}/corpus.jsonl.gz"):
    !wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz
    !mv corpus.jsonl.gz {DIRETORIO_TREC_COVID}
    print('Baixado')
else:
    print('Já existia a pasta')

Já existia a pasta


In [36]:
# Descompacte o arquivo para a memória
with gzip.open(f'{DIRETORIO_TREC_COVID}/corpus.jsonl.gz', 'rt') as f:
    # Leia o conteúdo do arquivo descompactado
    corpus = [json.loads(line) for line in f]

In [37]:
# Exiba os dados carregados
print(f"{type(corpus)} len(corpus): {len(corpus)} " )

<class 'list'> len(corpus): 171332 


In [38]:
corpus_dict = {}

for docto in corpus:
    if ('title' in docto) and len(docto['title']) >= 5:
        texto_usado_na_geracao_de_query = docto['title'] + '. ' + docto['text']
    else:
        texto_usado_na_geracao_de_query = docto['text']
    corpus_dict[docto['_id']] = {'text_query_generation': texto_usado_na_geracao_de_query, 
                                 'title': docto['title'],
                                 'text': docto['text']}

In [39]:
print(len(corpus_dict))

171332


## Queries

In [40]:
from pyserini.search import get_topics

In [41]:
topics = get_topics('covid-round5')
print(f'{len(topics)} queries total')

50 queries total


In [42]:
topics[1]

{'question': 'what is the origin of COVID-19',
 'query': 'coronavirus origin',
 'narrative': "seeking range of information about the SARS-CoV-2 virus's origin, including its evolution, animal source, and first transmission into humans"}

## Dados de relevância (qrel de teste)

In [43]:
if not os.path.exists(f'{DIRETORIO_TREC_COVID}/test.tsv'):
    !wget https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv
    !mv test.tsv {DIRETORIO_LOCAL}/
else:
    print('Arquivo já existia')

Arquivo já existia


In [44]:
qrel = pd.read_csv(f"{DIRETORIO_TREC_COVID}/test.tsv", sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rel"])

In [45]:
qrel.head()

,query,docid,rel
0,1,005b2j4b,2
1,1,00fmeepz,1
2,1,g7dhmyyo,2
3,1,0194oljo,1
4,1,021q9884,1


In [46]:
qrel.head()

,query,docid,rel
0,1,005b2j4b,2
1,1,00fmeepz,1
2,1,g7dhmyyo,2
3,1,0194oljo,1
4,1,021q9884,1


In [47]:
qrel["q0"] = "q0"
qrel_dict = qrel.to_dict(orient="list")

In [48]:
qrel_dict['query'][0], qrel_dict['docid'][0], qrel_dict['rel'][0]

(1, '005b2j4b', 2)

# Rotinas para cálculos e lógica a ser usada


Plano de ação:


        Para cada pipeline, calcular  e salvar

                indexação:
                        tempo 
                        memória

                retrieval: 
                        tempo_medio_por_query
                        memória

                resultado:
                        ndcg@10
        
                Detalhes de implementação:  salvar no dict resultado_pipeline com key o nome pipeline e values outro dict:
                        {
                         'tempo_indexacao_segundo': lista de tuplas, conforme abaixo:
                                        [{'tipo':'gpu'/'cpu', 'valor': float, segundos}], 
                         'memoria_indice_byte_ram'; float, em bytes
                         'retrieval_tempo_medio_por_query': [{'tipo':'gpu'/'cpu', 'valor': float, segundos}], 
                         'se_retrieval_usa_gpu': boolean
                         'ndcg_10': float formato 99,99}
                
                        sendo o tempo em segundos e memória em kbytes


        Criar método explora_contexto que a partir dos parâmetros:
                
                tipo_gpu
                        3090


                resultado_pipeline (passo anterior)


                calcula 
                        usd/query
                        usd/mês

                para os seguintes contextos: 
                        "utilizacao_perfeita" (assim que terminou de processar uma query, já tem outra para ser processada)
                        "utilizacao_precaria_100" (sistema precisar rodar 24h/7h, mas apenas 100 queries/dia)


                Detalhes de implementação: salva em um novo dict avaliacao_pipeline_contexto com key o nome do pipeline e value outro dict:
                {<nome contexto>: 
                        {'usd_query':, 
                         'usd_mes';, }
                }


        Testar os seguintes pipelines:

                bm25 
                bm25@100_reranking_minilm
                bm25@1000_reranking_minilm
                splade

                (se der tempo:)
                bm25@100_reranking_monot5_3b
                bm25@1000_reranking_monot5_3b
                dpr


                Detalhes de implementação: criar lista lista_pipeline com nomes acima


(pode usar 3090, mas estimar o custo para A100, considerar quantas vezes mais lentas: a100 tem x flops e custa Z, daí derivar o 3090)


        Velocidade do clock: 1,40 GHz (Boost Clock 1,70 GHz)
        Memória de vídeo (VRAM): 24 GB

https://versus.com/br/nvidia-geforce-rtx-3090-vs-nvidia-tesla-t4



    Por que Nvidia GeForce RTX 3090 é melhor que Nvidia Tesla T4?
        Clock do GPU 390MHz mais rápido?
            1395MHz vs 1005MHz
        Desempenho 27.82 TFLOPS maior de ponto flutuante?
            35.58 TFLOPS vs 7.76 TFLOPS
        Taxa de píxeis 92.84 GPixel/s maior?
            189.8 GPixel/s vs 96.96 GPixel/s
        Mais 8GB de VRAM?
            24GB vs 16GB
        Velocidade efetiva do clock da memória 9500MHz maior?
            19500MHz vs 10000MHz
        Taxa de textura 313.6 GTexels/s maior?
            556 GTexels/s vs 242.4 GTexels/s
        616GB/s mais largura de banda de memória?
            936GB/s vs 320GB/s
        Suporta ray tracing


Memórica de cálculo de custo de GPU 3090

Parâmetro de cálculo, site encontrado que tem tanto 3090 quanto uma das definidas no enunciado do exercício.


Em [https://cloud.vast.ai](https://cloud.vast.ai/create?utm_medium=cpc&utm_target=&device=c&gad=1&gclid=Cj0KCQjwr82iBhCuARIsAO0EAZxdEU16ZYFVO9asqO5T3yMYqs1Y3WL3YbHbFIVh-8p_b4Svp9k0HLYaAgUVEALw_wcB&utm_content=633581964120&placement=&utm_group=143233570036&adposition=&utm_source=google&utm_campaign=18841339102_search) 

        3090  (44.1tflops/24gb) 0.20 a 0.33. 
        v100 (14.8tflops/16gb) 2.30 a 3.06 
        a100 (19.5tflops/80gb) 1.55 a 1.8


Considerando que no enunciado diz:
        
        1,50 USD/hora por A100 ou 0,21 USD/hora por T4 ou 0,50 USD/hora por V100


Considerando a proporção de a100 para 3090 do site cloud.vast.ai: 6x

Assumiremos custo de 0,25 USD/hora

Mudando para segundo:

Regra de 3:
        3600 s       1s
        0,25         x

        x = 25e-2 / 3600 = 6,944...e-5


In [49]:
print( 25e-2 / 3600 )

6.944444444444444e-05


Algumas suposições para cálculo

Para fins de cálculo de memória ram, computaremos apenas o espaço ocupado pelo índice (o fator mais diferenciável entre os pipelines), descartando ocupações em memória por outros objetos e variáveis


Outros valores definidos no enunciado - transformando

        0,03 USD/hora por CPU core
        0,005 USD/hora por GB de CPU RAM




Mudando para bytes:


In [50]:
print( 0.005 / 2**30)

4.656612873077393e-12


Constantes de valores

In [51]:
custo_gpu_segundo = {'3090': 6.944e-5}

In [52]:
CUSTO_RAM_CPU_HORA_BYTE = 4.656e-12

In [53]:
CUSTO_CPU_ALOCADA_HORA = 0.03



In [54]:
print(CUSTO_CPU_ALOCADA_HORA / 3600)

8.333333333333334e-06


In [55]:
CUSTO_CPU_ALOCADA_SEGUNDO = 8.333e-6


Variáveis usadas

In [56]:
lista_pipeline = [  'bm25',
                    'bm25@100_reranking_minilm',
                    'bm25@1000_reranking_minilm',
                    'bm25@100_reranking_monot5_3b',
                    'bm25@1000_reranking_monot5_3b',
                    'inpars',
                    ]

In [57]:
resultado_pipeline = {}
avaliacao_pipeline_contexto = {}

## Funções de apoio

In [58]:
def retorna_calculo_contexto (parm_resultado_pipeline: dict,
                      parm_tipo_gpu:str='3090',
                      ):
    """
      Método para efetuar cálculos de gastos conforme
        parm_tipo_gpu
        parm_resultado_pipeline
     
    """ 
    global lista_pipeline 
    assert parm_tipo_gpu in ['3090'], f"parm_tipo_gpu {parm_tipo_gpu} não está previsto para cálculos!"

    resultado = {}
    for contexto in ['utilizacao_perfeita', 'utilizacao_precaria_100']:
        avaliacao_contexto = calcula_gastos(parm_resultado_pipeline, parm_tipo_gpu, contexto)
        resultado[contexto]= avaliacao_contexto
    return resultado
    




In [59]:
def calcula_gastos (parm_dados: dict,
                      parm_tipo_gpu:str='3090',
                      parm_contexto:str='utilizacao_perfeita'):
    """
        parm_dados deve ter:
                        {'tempo_indexacao_segundo':, lista de tuplas, conforme abaixo:
                                        [{'tipo':'gpu'/'cpu', 'valor': float, segundos}], 
                         'memoria_indice_byte_ram';, 
                         'retrieval_tempo_medio_por_query':,
                         'se_retrieval_usa_gpu':}

    """
    global custo_gpu_segundo, CUSTO_RAM_CPU_HORA_BYTE, CUSTO_CPU_ALOCADA_HORA, CUSTO_CPU_ALOCADA_SEGUNDO
    lista_chave_esperada = ['tempo_indexacao_segundo','memoria_indice_byte_ram','retrieval_tempo_medio_por_query','se_retrieval_usa_gpu']
    assert parm_tipo_gpu in ['3090'], f"parm_tipo_gpu {parm_tipo_gpu} não está previsto para cálculos!"
    assert parm_contexto in ['utilizacao_perfeita', 'utilizacao_precaria_100'], f"parm_contexto {parm_contexto} não está previsto para cálculos!"
    
    for chave in lista_chave_esperada:
        assert chave in parm_dados, f"chave {chave} não está em parm_dados {parm_dados} para os cálculos!"

    dict_retorno = {}


    # custo indexacao
    custo_indexacao_tempo = 0

    ## acumular custo por tempo
    for tempo_valor in parm_dados['tempo_indexacao_segundo']:
        if tempo_valor['tipo'] == 'cpu':
            custo_indexacao_tempo +=  tempo_valor['valor'] * CUSTO_CPU_ALOCADA_SEGUNDO 
        elif tempo_valor['tipo'] == 'gpu':
            custo_indexacao_tempo +=  tempo_valor['valor'] * CUSTO_CPU_ALOCADA_SEGUNDO
        else:
            raise Exception(f"Tipo de tempo deveria ser cpu ou gpu e não {tempo_valor['tipo']}")


    # tem que deixar cpu disponível 24h
    custo_cpu_dia = 24 * CUSTO_CPU_ALOCADA_HORA

    # índice tem que ficar em memória
    custo_memoria_dia = 24 * parm_dados['memoria_indice_byte_ram'] * CUSTO_RAM_CPU_HORA_BYTE

    custo_dia = custo_memoria_dia + custo_cpu_dia

    custo_gpu_dia = 0

    if parm_contexto == 'utilizacao_perfeita': #(assim que terminou de processar uma query, já tem outra para ser processada)

        if parm_dados['se_retrieval_usa_gpu']:
            custo_gpu_dia = 24 * 3600 * custo_gpu_segundo[parm_tipo_gpu]
            custo_dia += custo_gpu_dia 
            print(f"para {parm_contexto} custo gpu dia: {custo_gpu_dia}")

        num_queries_dia = (24 * 3600) / parm_dados['retrieval_tempo_medio_por_query']
        custo_query = round(custo_dia / num_queries_dia, 10)

    elif parm_contexto == 'utilizacao_precaria_100': #(assim que terminou de processar uma query, já tem outra para ser processada)

        if parm_dados['se_retrieval_usa_gpu']:
            custo_gpu_dia = 100 * parm_dados['retrieval_tempo_medio_por_query'] * custo_gpu_segundo[parm_tipo_gpu]
            custo_dia += custo_gpu_dia 
            print(f"para {parm_contexto} custo gpu dia: {custo_gpu_dia}")

        num_queries_dia = 100
        custo_query = round(custo_dia / num_queries_dia, 10)

    return {'usd_query': custo_query, 
            'usd_dia': custo_dia,
            'usd_gpu_dia': custo_gpu_dia,
            'usd_mes': (30 * custo_dia),
            'usd_indexacao_tempo': custo_indexacao_tempo, 
             }


# Realiza buscas com os pipelines

In [60]:
trec_eval = load("trec_eval")

# Pipeline BM25

## Objetos de apoio

In [61]:
nome_pipeline = 'bm25'

In [62]:
# Run all queries in topics, retrive top 1k for each query
def run_all_queries_bm25(file, topics, searcher, num_max_hits=100):
  """
  A função run_all_queries é responsável por realizar todas as consultas armazenadas no dicionário topics utilizando o objeto searcher fornecido e salvar os resultados em um arquivo de texto.
  Usada no notebook da aula 2

  Parâmetros:

  file: caminho do arquivo de saída onde serão salvos os resultados das consultas.
  topics: dicionário contendo as consultas a serem executadas. Cada consulta é representada por uma chave única no dicionário. O valor correspondente a cada chave é um outro dicionário contendo as informações da consulta, como seu título e outras informações relevantes.
  searcher: objeto do tipo Searcher que será utilizado para realizar as consultas.
  num_max_hits: número máximo de documentos relevantes que serão retornados para cada consulta.
  Retorno:

  A função não retorna nenhum valor, mas salva os resultados das consultas no arquivo especificado em file.
  Comentário:

  A função usa a biblioteca tqdm para exibir uma barra de progresso enquanto executa as consultas.
  O número de consultas concluídas é impresso a cada 100 consultas.
  """
  tempos = []
  print(f'Running {len(topics)} queries in total')
  with open(file, 'w') as runfile:
    cnt = 0
    for id in tqdm(topics, desc='Running Queries'):
        # print(f'id = {id}')
        query = topics[id]['question']
        # print(f'query = {query}')
        tempo_inicio = time.time()
        hits = searcher.search(query, num_max_hits)
        tempos.append(time.time() - tempo_inicio)

        for i in range(0, len(hits)):
            _ = runfile.write(f'{id} Q0 {hits[i].docid} {i+1} {hits[i].score:.6f} Busca\n')
            # = runfile.write('{} Q0 {} {} {:.6f} Pyserini\n'.format(id, hits[i].docid, i+1, hits[i].score))
        cnt += 1
        if cnt % 100 == 0:
            print(f'{cnt} queries completed')
  return tempos


In [63]:
searcher = LuceneSearcher(PATH_INDICE_TRECC) # './indexes/lucene-index-msmarco-passage')

## Parâmetros k1=1.12, b=0.4

In [64]:
num_max_hits = 1000

In [65]:
resultado_execucao = {}

In [66]:
resultado_execucao['tempo_indexacao_segundo'] = [{'tipo':'cpu', 'valor': tempo_construcao_indice_trecc_pyserine}]
resultado_execucao['memoria_indice_byte_ram'] = tamanho_em_bytes_indice_trecc
resultado_execucao['se_retrieval_usa_gpu'] = False

In [67]:
searcher.set_bm25(k1=1.12, b=0.4)    # valor sugerido pelo Dr Carísio em seu grid na tarefa Doc2Query

In [68]:
tempo_gasto = run_all_queries_bm25(PATH_RUN_AVALIACAO, topics, searcher, num_max_hits)

Running 50 queries in total


Running Queries: 100%|██████████| 50/50 [00:02<00:00, 21.65it/s]


In [69]:
df_tempos = pd.DataFrame({'tempo_gasto': tempo_gasto})
df_tempos['tempo_gasto'].describe()

count    50.000000
mean      0.044992
std       0.018053
min       0.033016
25%       0.037938
50%       0.039992
75%       0.046751
max       0.155465
Name: tempo_gasto, dtype: float64

In [70]:
round(df_tempos['tempo_gasto'].describe()['mean'],6)

0.044992

In [71]:
resultado_execucao['retrieval_tempo_medio_por_query'] = round(df_tempos['tempo_gasto'].describe()['mean'],6)

In [72]:
### Calculando métricas
run = pd.read_csv(f"{PATH_RUN_AVALIACAO}", sep="\s+", 
                names=["query", "q0", "docid", "rank", "score", "system"])
print(run.head())
run = run.to_dict(orient="list")
results = trec_eval.compute(predictions=[run], references=[qrel_dict])
# salvando métricas    
print(f"NDCG@10: {results['NDCG@10']}")
print(f"Resultados: {results}")

   query  q0     docid  rank      score system
0     44  Q0  xfjexm5b     1  11.889200  Busca
1     44  Q0  28utunid     2  10.906200  Busca
2     44  Q0  qi1henyy     3  10.906199  Busca
3     44  Q0  ugkxxaeb     4  10.638000  Busca
4     44  Q0  qp77vl6h     5  10.487900  Busca
NDCG@10: 0.6187936694210939
Resultados: {'runid': 'Busca', 'num_ret': 50000, 'num_rel': 24673, 'num_rel_ret': 9804, 'num_q': 50, 'map': 0.19397103009765435, 'gm_map': 0.13129328791759654, 'bpref': 0.334829488236771, 'Rprec': 0.28961812585830815, 'recip_rank': 0.8518571428571429, 'P@5': 0.7080000000000001, 'P@10': 0.67, 'P@15': 0.6453333333333333, 'P@20': 0.6199999999999999, 'P@30': 0.5946666666666667, 'P@100': 0.48460000000000003, 'P@200': 0.4014000000000001, 'P@500': 0.28308, 'P@1000': 0.19608, 'NDCG@5': 0.6494195661372965, 'NDCG@10': 0.6187936694210939, 'NDCG@15': 0.5930265595572919, 'NDCG@20': 0.5686991493574204, 'NDCG@30': 0.5424134242812346, 'NDCG@100': 0.45176650622218156, 'NDCG@200': 0.3909233482135728

In [73]:
resultado_execucao['ndcg_10'] = round(100*results['NDCG@10'],2)

In [74]:
resultado_pipeline[nome_pipeline] = resultado_execucao
print(f"resultado_pipeline[{nome_pipeline}] {resultado_pipeline[nome_pipeline]}")

resultado_pipeline[bm25] {'tempo_indexacao_segundo': [{'tipo': 'cpu', 'valor': 0.109634}], 'memoria_indice_byte_ram': 269772727, 'se_retrieval_usa_gpu': False, 'retrieval_tempo_medio_por_query': 0.044992, 'ndcg_10': 61.88}


In [75]:
avaliacao_pipeline_contexto[nome_pipeline] = retorna_calculo_contexto(resultado_execucao, parm_tipo_gpu='3090')
print(avaliacao_pipeline_contexto[nome_pipeline] )

{'utilizacao_perfeita': {'usd_query': 3.906e-07, 'usd_dia': 0.750145483605888, 'usd_gpu_dia': 0, 'usd_mes': 22.50436450817664, 'usd_indexacao_tempo': 9.135801219999999e-07}, 'utilizacao_precaria_100': {'usd_query': 0.0075014548, 'usd_dia': 0.750145483605888, 'usd_gpu_dia': 0, 'usd_mes': 22.50436450817664, 'usd_indexacao_tempo': 9.135801219999999e-07}}


# Avaliações de pipelines com estágio 1 bm25 e estágio 2 reranking

## Objetos de apoio

In [133]:
# Run all queries in topics, retrive top 1k for each query
def run_all_queries_bm25_reranking(file, topics, searcher, parm_model, parm_tokenizer, parm_model_name:str, num_max_hits=100):
  """
  A função run_all_queries é responsável por realizar todas as consultas armazenadas no dicionário topics utilizando o objeto searcher fornecido e salvar os resultados em um arquivo de texto.
  Usada no notebook da aula 2

  Parâmetros:

  file: caminho do arquivo de saída onde serão salvos os resultados das consultas.
  topics: dicionário contendo as consultas a serem executadas. Cada consulta é representada por uma chave única no dicionário. O valor correspondente a cada chave é um outro dicionário contendo as informações da consulta, como seu título e outras informações relevantes.
  searcher: objeto do tipo Searcher que será utilizado para realizar as consultas.
  num_max_hits: número máximo de documentos relevantes que serão retornados para cada consulta.
  Retorno:

  A função não retorna nenhum valor, mas salva os resultados das consultas no arquivo especificado em file.
  Comentário:

  A função usa a biblioteca tqdm para exibir uma barra de progresso enquanto executa as consultas.
  O número de consultas concluídas é impresso a cada 100 consultas.
  """
  tempos = []
  print(f'Running {len(topics)} queries in total')
  with open(file, 'w') as runfile:
    cnt = 0
    for id in tqdm(topics, desc='Running Queries'):
        query = topics[id]['question']
        
        tempo_inicio = time.time()
        hits = searcher.search(query, num_max_hits)
        # reranking
        docids = [hit.docid for hit in hits]
        pares_texto = [(query, corpus_dict[docid]['text_query_generation']) for docid in docids]

        # print("carregando dataset")  
        classes_dummy = np.zeros(len(hits), dtype=np.int64)
        dataset_reranking = MyDataset(texts=pares_texto, classes=classes_dummy, tokenizer=parm_tokenizer)    
        dataloader_reranking = DataLoader(dataset_reranking,
                                          batch_size= 16,
                                          shuffle=False)
        prob_relevancia = calcula_relevancia(parm_model,dataloader_reranking, parm_model_name = parm_model_name)
        resultado_ordenado = sorted(zip(docids, prob_relevancia), key=lambda x: x[1], reverse=True)
        tempos.append(time.time() - tempo_inicio)

        for i in range(0, len(hits)):
            _ = runfile.write(f'{id} Q0 {resultado_ordenado[i][0]} {i+1} {resultado_ordenado[i][1]:.6f} Busca\n')
        cnt += 1
        if cnt % 100 == 0:
            print(f'{cnt} queries completed')
  return tempos


In [134]:
class MyDataset(Dataset):
    """
      Classe para representar um dataset de texto e classes.
    """  
    def __init__(self, texts: list, classes:list[int], tokenizer):
      """
      Inicializa um novo objeto MyDataset.

      Args:
          texts (list): uma lista com as strings de texto. Cada elemento deve ter 2 strings.
          classes (np.ndarray): um array com as classes de cada texto.
          tokenizer: um objeto tokenizer do Hugging Face Transformers.
          max_seq_length (int): o tamanho máximo da sequência a ser considerado.
      Raises:
          AssertionError: se os parâmetros não estiverem no formato esperado.
      """
      # Verifica se os parâmetros são do tipo esperado
      assert isinstance(texts, list), f"Parâmetro texts deve ser do tipo list e não {type(texts)}"
      for row in texts:
          assert isinstance(row, tuple) and len(row)== 2, f"Each row in texts must have 2 elements"
          assert isinstance(row[0], str) and isinstance(row[1], str), f"Each element in texts.row must be a string e não {type(row[0])}"
      assert isinstance(classes,np.ndarray), f'classes deve ser do tipo np.ndarray e não {type(classes)}'
      assert isinstance(classes[0],np.int64), f'classes[0] deve ser do tipo numpy.int64 e não {type(classes[0])} '

      self.texts = texts
      self.classes = classes
      self.tokenizer = tokenizer
      self.max_seq_length = tokenizer.model_max_length # model.config.max_position_embeddings
      if self.max_seq_length > 64000:
        print(f"Valor de self.max_seq_length  {self.max_seq_length} indica que deve ser usado outro campo do tokenizador. Assumido 512 ")
        self.max_seq_length =  512
      # Salvar os dados dos tensores
      x_data_input_ids = []
      x_data_token_type_ids = []
      x_data_attention_masks = []
      for text_pair in texts:
          encoding = tokenizer.encode_plus(
              text_pair[0],
              text_pair[1],
              add_special_tokens=True,
              max_length=self.max_seq_length,
              padding='max_length',
              return_tensors = 'pt',
              truncation=True,
              return_attention_mask=True,
              return_token_type_ids=True
          )
          x_data_input_ids.append(encoding['input_ids'].long())
          x_data_token_type_ids.append(encoding['token_type_ids'].long())
          x_data_attention_masks.append(encoding['attention_mask'].long())
      # print(F'\tVou converter lista para tensor;  Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")}')
      # squeeze: vai transformar um tensor de shape [2, 1, 322] em um tensor de shape [2, 322].

      self.x_tensor_input_ids = torch.stack(x_data_input_ids).squeeze(1)
      self.x_tensor_attention_masks = torch.stack(x_data_attention_masks).squeeze(1)
      self.x_tensor_token_type_ids = torch.stack(x_data_token_type_ids).squeeze(1)

    def __len__(self):
        """
          Retorna o tamanho do dataset (= tamanho do array texts)
        """
        return len(self.texts)
    
    def __getitem__(self, idx):
        """
          Retorna um dicionário com os dados do texto e sua classe correspondente, em um formato que pode 
          ser usado pelo dataloader do PyTorch para alimentar um modelo de aprendizado de máquina.
        """
        return {
            'input_ids': self.x_tensor_input_ids[idx],
            'attention_mask': self.x_tensor_attention_masks[idx],
            'token_type_ids': self.x_tensor_token_type_ids[idx],
            # 'labels': int(self.classes[idx])
            'labels': torch.tensor(self.classes[idx], dtype=torch.long)            
        }

In [140]:
def calcula_relevancia(parm_model, parm_dataloader_reranking, parm_model_name:str):
  # para 'cross-encoder/ms-marco-TinyBERT-L-2'
  prob_relevancia = []
  parm_model.eval()
  with torch.no_grad():
      for ndx, batch in enumerate(parm_dataloader_reranking):
          if 'minilm' in parm_model_name:
              logits_model = parm_model(**BatchEncoding(batch).to(device)).logits                          
              relevantes_float = [float(t) for t in logits_model]
          elif 'monobert' in parm_model_name:
              logits_model = parm_model(**BatchEncoding(batch).to(device)).logits
              probs = torch.nn.functional.softmax(logits_model, dim=1)
              nao_relevante, relevante = zip(*probs)
              relevantes_float = [float(t) for t in relevante]              
          prob_relevancia.extend(relevantes_float)          
          # prob_relevancia.append(pa.array(scores.numpy()))
  return prob_relevancia

In [ ]:
def calcula_relevancia_softmax(parm_model, parm_dataloader_reranking):
  prob_relevancia = []
  parm_model.eval()
  with torch.no_grad():
      for ndx, batch in tqdm(enumerate(parm_dataloader_reranking), total=len(parm_dataloader_reranking), mininterval=0.5, desc='dataset_reranking', disable=False):
          #print("\nbatch['input_ids'][0]", batch['input_ids'][0])
          #print("batch['input_ids'][1]", batch['input_ids'][1])
          logits_model = parm_model(**BatchEncoding(batch).to(device)).logits
          probs = torch.nn.functional.softmax(logits_model, dim=1)
          nao_relevante, relevante = zip(*probs)
          relevantes_float = [float(t) for t in relevante]
          # print('logits_model', logits_model)
          prob_relevancia.extend(relevantes_float)
          # prob_relevancia.append(pa.array(scores.numpy()))
          # print('probs',probs)
          # print('relevantes_float',relevantes_float)
          # break
  return prob_relevancia

def calcula_relevancia_monobert(parm_model, parm_dataloader_reranking):
  # para 'cross-encoder/ms-marco-TinyBERT-L-2'
  prob_relevancia = []
  parm_model.eval()
  with torch.no_grad():
      for ndx, batch in enumerate(parm_dataloader_reranking):
          outputs, = parm_model(**BatchEncoding(batch).to(device))                         
          relevantes_float = [float(t) for t in outputs]
          prob_relevancia.extend(relevantes_float)          
          # prob_relevancia.append(pa.array(scores.numpy()))
  return prob_relevancia

                output, = self.model(input_ids, token_type_ids=tt_ids, return_dict=False)
                if output.size(1) > 1:
                    text.score = torch.nn.functional.log_softmax(
                        output, 1)[0, -1].item()
                else:
                    text.score = output.item()

# Reranking com miniLM (treinado no ms-marco)

In [79]:
# nome_modelo = 'cross-encoder/ms-marco-TinyBERT-L-2'
nome_modelo = 'cross-encoder/ms-marco-MiniLM-L-6-v2'
model = AutoModelForSequenceClassification.from_pretrained(nome_modelo).to(device)
tokenizer = AutoTokenizer.from_pretrained(nome_modelo)

In [80]:
model.config.max_position_embeddings, tokenizer.max_len_sentences_pair, tokenizer.model_max_length

(512, 509, 512)

## bm25@100_reranking_minilm

In [81]:
nome_pipeline = 'bm25@100_reranking_minilm'

In [82]:
resultado_execucao = {}

In [83]:
resultado_execucao['tempo_indexacao_segundo'] = [{'tipo':'cpu', 'valor': tempo_construcao_indice_trecc_pyserine}]
resultado_execucao['memoria_indice_byte_ram'] = tamanho_em_bytes_indice_trecc
resultado_execucao['se_retrieval_usa_gpu'] = True

In [84]:
tempo_gasto = run_all_queries_bm25_reranking(PATH_RUN_AVALIACAO, topics, searcher, model, tokenizer, num_max_hits=100)

Running 50 queries in total


Running Queries: 100%|██████████| 50/50 [00:14<00:00,  3.57it/s]


In [85]:
df_tempos = pd.DataFrame({'tempo_gasto': tempo_gasto})
resultado_execucao['retrieval_tempo_medio_por_query'] = round(df_tempos['tempo_gasto'].describe()['mean'],6)
print(f"Tempo médio por query {resultado_execucao['retrieval_tempo_medio_por_query']}")

Tempo médio por query 0.279581


In [86]:
### Calculando métricas
run = pd.read_csv(f"{PATH_RUN_AVALIACAO}", sep="\s+", 
                names=["query", "q0", "docid", "rank", "score", "system"])
print(run.head())
run = run.to_dict(orient="list")

   query  q0     docid  rank     score system
0     44  Q0  qi1henyy     1  6.995133  Busca
1     44  Q0  28utunid     2  6.964627  Busca
2     44  Q0  ej76fsxa     3  6.382739  Busca
3     44  Q0  uc37poce     4  6.346030  Busca
4     44  Q0  dt2pew66     5  6.306264  Busca


In [87]:
results = trec_eval.compute(predictions=[run], references=[qrel_dict])
# salvando métricas    
print(f"NDCG@10: {results['NDCG@10']}")
print(f"Resultados: {results}")


NDCG@10: 0.7484831661234049
Resultados: {'runid': 'Busca', 'num_ret': 5000, 'num_rel': 24673, 'num_rel_ret': 2423, 'num_q': 50, 'map': 0.0877165751524934, 'gm_map': 0.06848099815752387, 'bpref': 0.10889678899977348, 'Rprec': 0.11195682644552621, 'recip_rank': 0.8745238095238095, 'P@5': 0.836, 'P@10': 0.8100000000000002, 'P@15': 0.7973333333333333, 'P@20': 0.7660000000000001, 'P@30': 0.7146666666666668, 'P@100': 0.48460000000000003, 'P@200': 0.24230000000000002, 'P@500': 0.09692, 'P@1000': 0.04846, 'NDCG@5': 0.7716916005180048, 'NDCG@10': 0.7484831661234049, 'NDCG@15': 0.7305901802777125, 'NDCG@20': 0.7090515248486612, 'NDCG@30': 0.6680300786452001, 'NDCG@100': 0.4800372821877119, 'NDCG@200': 0.29914468380869985, 'NDCG@500': 0.1991620051549731, 'NDCG@1000': 0.1875552589200808}


In [88]:
resultado_execucao['ndcg_10'] = round(100*results['NDCG@10'],2)

In [89]:
resultado_pipeline[nome_pipeline] = resultado_execucao
print(resultado_pipeline[nome_pipeline])

{'tempo_indexacao_segundo': [{'tipo': 'cpu', 'valor': 0.109634}], 'memoria_indice_byte_ram': 269772727, 'se_retrieval_usa_gpu': True, 'retrieval_tempo_medio_por_query': 0.279581, 'ndcg_10': 74.85}


In [90]:
avaliacao_pipeline_contexto[nome_pipeline] = retorna_calculo_contexto(resultado_execucao, parm_tipo_gpu='3090')
print(avaliacao_pipeline_contexto[nome_pipeline] )

para utilizacao_perfeita custo gpu dia: 5.999616
para utilizacao_precaria_100 custo gpu dia: 0.001941410464
{'utilizacao_perfeita': {'usd_query': 2.18415e-05, 'usd_dia': 6.749761483605887, 'usd_gpu_dia': 5.999616, 'usd_mes': 202.49284450817663, 'usd_indexacao_tempo': 9.135801219999999e-07}, 'utilizacao_precaria_100': {'usd_query': 0.0075208689, 'usd_dia': 0.7520868940698879, 'usd_gpu_dia': 0.001941410464, 'usd_mes': 22.562606822096637, 'usd_indexacao_tempo': 9.135801219999999e-07}}


## bm25@500_reranking_minilm

In [91]:
nome_pipeline = 'bm25@500_reranking_minilm'

In [104]:
resultado_execucao = {}

In [105]:
resultado_execucao['tempo_indexacao_segundo'] = [{'tipo':'cpu', 'valor': tempo_construcao_indice_trecc_pyserine}]
resultado_execucao['memoria_indice_byte_ram'] = tamanho_em_bytes_indice_trecc
resultado_execucao['se_retrieval_usa_gpu'] = True

In [106]:
tempo_gasto = run_all_queries_bm25_reranking(PATH_RUN_AVALIACAO, topics, searcher, model, tokenizer, num_max_hits=500)

Running 50 queries in total


Running Queries: 100%|██████████| 50/50 [01:07<00:00,  1.35s/it]


In [107]:
df_tempos = pd.DataFrame({'tempo_gasto': tempo_gasto})
resultado_execucao['retrieval_tempo_medio_por_query'] = round(df_tempos['tempo_gasto'].describe()['mean'],6)
print(f"Tempo médio por query {resultado_execucao['retrieval_tempo_medio_por_query']}")

Tempo médio por query 1.347039


In [108]:
### Calculando métricas
run = pd.read_csv(f"{PATH_RUN_AVALIACAO}", sep="\s+", 
                names=["query", "q0", "docid", "rank", "score", "system"])
print(run.head())
run = run.to_dict(orient="list")

results = trec_eval.compute(predictions=[run], references=[qrel_dict])
# salvando métricas    
print(f"NDCG@10: {results['NDCG@10']}")
print(f"Resultados: {results}")

   query  q0     docid  rank     score system
0     44  Q0  qi1henyy     1  6.995133  Busca
1     44  Q0  28utunid     2  6.964627  Busca
2     44  Q0  ej76fsxa     3  6.382739  Busca
3     44  Q0  uc37poce     4  6.346030  Busca
4     44  Q0  dt2pew66     5  6.306264  Busca
NDCG@10: 0.7146676971972016
Resultados: {'runid': 'Busca', 'num_ret': 25000, 'num_rel': 24673, 'num_rel_ret': 7077, 'num_q': 50, 'map': 0.18720813343505024, 'gm_map': 0.14347464567182966, 'bpref': 0.2724897873211263, 'Rprec': 0.2772609408960807, 'recip_rank': 0.87, 'P@5': 0.8039999999999999, 'P@10': 0.774, 'P@15': 0.772, 'P@20': 0.7450000000000001, 'P@30': 0.7213333333333334, 'P@100': 0.5663999999999999, 'P@200': 0.4569, 'P@500': 0.28308, 'P@1000': 0.14154, 'NDCG@5': 0.7370341620293666, 'NDCG@10': 0.7146676971972016, 'NDCG@15': 0.706085483631409, 'NDCG@20': 0.6825016095070311, 'NDCG@30': 0.6640399915178563, 'NDCG@100': 0.5384526958818792, 'NDCG@200': 0.4541754023619141, 'NDCG@500': 0.3820112923070092, 'NDCG@1000': 

Salvando resultados

In [109]:
resultado_execucao['ndcg_10'] = round(100*results['NDCG@10'],2)

In [110]:
resultado_pipeline[nome_pipeline] = resultado_execucao
print(resultado_pipeline[nome_pipeline])

{'tempo_indexacao_segundo': [{'tipo': 'cpu', 'valor': 0.109634}], 'memoria_indice_byte_ram': 269772727, 'se_retrieval_usa_gpu': True, 'retrieval_tempo_medio_por_query': 1.347039, 'ndcg_10': 71.47}


In [113]:
avaliacao_pipeline_contexto[nome_pipeline] = retorna_calculo_contexto(resultado_execucao, parm_tipo_gpu='3090')
print(avaliacao_pipeline_contexto[nome_pipeline] )

para utilizacao_perfeita custo gpu dia: 5.999616
para utilizacao_precaria_100 custo gpu dia: 0.009353838816
{'utilizacao_perfeita': {'usd_query': 0.0001052337, 'usd_dia': 6.749761483605887, 'usd_gpu_dia': 5.999616, 'usd_mes': 202.49284450817663, 'usd_indexacao_tempo': 9.135801219999999e-07}, 'utilizacao_precaria_100': {'usd_query': 0.0075949932, 'usd_dia': 0.759499322421888, 'usd_gpu_dia': 0.009353838816, 'usd_mes': 22.78497967265664, 'usd_indexacao_tempo': 9.135801219999999e-07}}


# Reranking com monobert (treinado no ms-marco)

In [116]:
# from transformers import AutoModelForSeq2SeqLM
# nome_modelo = 'zeta-alpha-ai/monot5-3b-inpars-v2-hotpotqa'
# model = AutoModelForSeq2SeqLM.from_pretrained(nome_modelo).to(device)

In [121]:
nome_modelo = 'castorini/monobert-large-msmarco'
model = AutoModelForSequenceClassification.from_pretrained(nome_modelo).to(device)
tokenizer = AutoTokenizer.from_pretrained(nome_modelo)

In [122]:
model.config.max_position_embeddings, tokenizer.max_len_sentences_pair, tokenizer.model_max_length

(512, 509, 512)

Com num_max_hits=100, deu 2.61s por query

## bm25@50_reranking_monobert

In [143]:
nome_pipeline = 'bm25@50_reranking_monobert'

In [144]:
resultado_execucao = {}

In [145]:
resultado_execucao['tempo_indexacao_segundo'] = [{'tipo':'cpu', 'valor': tempo_construcao_indice_trecc_pyserine}]
resultado_execucao['memoria_indice_byte_ram'] = tamanho_em_bytes_indice_trecc
resultado_execucao['se_retrieval_usa_gpu'] = True

In [146]:
tempo_gasto = run_all_queries_bm25_reranking(PATH_RUN_AVALIACAO, topics, searcher, model, tokenizer, parm_model_name = nome_pipeline, num_max_hits=50)

Running 50 queries in total


Running Queries: 100%|██████████| 50/50 [01:02<00:00,  1.26s/it]


In [147]:
df_tempos = pd.DataFrame({'tempo_gasto': tempo_gasto})
resultado_execucao['retrieval_tempo_medio_por_query'] = round(df_tempos['tempo_gasto'].describe()['mean'],6)
print(f"Tempo médio por query {resultado_execucao['retrieval_tempo_medio_por_query']}")

Tempo médio por query 1.255988


In [148]:
### Calculando métricas
run = pd.read_csv(f"{PATH_RUN_AVALIACAO}", sep="\s+", 
                names=["query", "q0", "docid", "rank", "score", "system"])
print(run.head())
run = run.to_dict(orient="list")

   query  q0     docid  rank     score system
0     44  Q0  1c3fpazy     1  0.999238  Busca
1     44  Q0  tfrawa9z     2  0.999225  Busca
2     44  Q0  qi8x5yaq     3  0.999223  Busca
3     44  Q0  ej76fsxa     4  0.999214  Busca
4     44  Q0  dt2pew66     5  0.999213  Busca


In [149]:
results = trec_eval.compute(predictions=[run], references=[qrel_dict])
# salvando métricas    
print(f"NDCG@10: {results['NDCG@10']}")
print(f"Resultados: {results}")


NDCG@10: 0.7096526870747231
Resultados: {'runid': 'Busca', 'num_ret': 2500, 'num_rel': 24673, 'num_rel_ret': 1376, 'num_q': 50, 'map': 0.05166934942623486, 'gm_map': 0.04099062092032248, 'bpref': 0.0645296401936393, 'Rprec': 0.06572166044632141, 'recip_rank': 0.9216666666666667, 'P@5': 0.828, 'P@10': 0.7620000000000001, 'P@15': 0.7146666666666667, 'P@20': 0.695, 'P@30': 0.6519999999999999, 'P@100': 0.2752, 'P@200': 0.1376, 'P@500': 0.05504, 'P@1000': 0.02752, 'NDCG@5': 0.766001702045763, 'NDCG@10': 0.7096526870747231, 'NDCG@15': 0.670553196225495, 'NDCG@20': 0.6499719430381152, 'NDCG@30': 0.6095907287833424, 'NDCG@100': 0.3281946867628143, 'NDCG@200': 0.20559705151099014, 'NDCG@500': 0.13860890962768477, 'NDCG@1000': 0.13095453971365084}


In [150]:
resultado_execucao['ndcg_10'] = round(100*results['NDCG@10'],2)

In [151]:
resultado_pipeline[nome_pipeline] = resultado_execucao
print(resultado_pipeline[nome_pipeline])

{'tempo_indexacao_segundo': [{'tipo': 'cpu', 'valor': 0.109634}], 'memoria_indice_byte_ram': 269772727, 'se_retrieval_usa_gpu': True, 'retrieval_tempo_medio_por_query': 1.255988, 'ndcg_10': 70.97}


In [152]:
avaliacao_pipeline_contexto[nome_pipeline] = retorna_calculo_contexto(resultado_execucao, parm_tipo_gpu='3090')
print(avaliacao_pipeline_contexto[nome_pipeline] )

para utilizacao_perfeita custo gpu dia: 5.999616
para utilizacao_precaria_100 custo gpu dia: 0.008721580672
{'utilizacao_perfeita': {'usd_query': 9.81206e-05, 'usd_dia': 6.749761483605887, 'usd_gpu_dia': 5.999616, 'usd_mes': 202.49284450817663, 'usd_indexacao_tempo': 9.135801219999999e-07}, 'utilizacao_precaria_100': {'usd_query': 0.0075886706, 'usd_dia': 0.758867064277888, 'usd_gpu_dia': 0.008721580672, 'usd_mes': 22.76601192833664, 'usd_indexacao_tempo': 9.135801219999999e-07}}


## bm25@80_reranking_monobert

In [158]:
nome_pipeline = 'bm25@70_reranking_monobert'

In [159]:
resultado_execucao = {}

In [160]:
resultado_execucao['tempo_indexacao_segundo'] = [{'tipo':'cpu', 'valor': tempo_construcao_indice_trecc_pyserine}]
resultado_execucao['memoria_indice_byte_ram'] = tamanho_em_bytes_indice_trecc
resultado_execucao['se_retrieval_usa_gpu'] = True

In [161]:
tempo_gasto = run_all_queries_bm25_reranking(PATH_RUN_AVALIACAO, topics, searcher, model, tokenizer, parm_model_name = nome_pipeline, num_max_hits=70)

Running 50 queries in total


Running Queries: 100%|██████████| 50/50 [01:26<00:00,  1.72s/it]


In [162]:
df_tempos = pd.DataFrame({'tempo_gasto': tempo_gasto})
resultado_execucao['retrieval_tempo_medio_por_query'] = round(df_tempos['tempo_gasto'].describe()['mean'],6)
print(f"Tempo médio por query {resultado_execucao['retrieval_tempo_medio_por_query']}")

Tempo médio por query 1.722126


In [163]:
### Calculando métricas
run = pd.read_csv(f"{PATH_RUN_AVALIACAO}", sep="\s+", 
                names=["query", "q0", "docid", "rank", "score", "system"])
print(run.head())
run = run.to_dict(orient="list")

   query  q0     docid  rank     score system
0     44  Q0  sqr11fpv     1  0.999256  Busca
1     44  Q0  1c3fpazy     2  0.999238  Busca
2     44  Q0  tfrawa9z     3  0.999225  Busca
3     44  Q0  qi8x5yaq     4  0.999223  Busca
4     44  Q0  ej76fsxa     5  0.999214  Busca


In [164]:
results = trec_eval.compute(predictions=[run], references=[qrel_dict])
# salvando métricas    
print(f"NDCG@10: {results['NDCG@10']}")
print(f"Resultados: {results}")


NDCG@10: 0.7125345158714298
Resultados: {'runid': 'Busca', 'num_ret': 3500, 'num_rel': 24673, 'num_rel_ret': 1809, 'num_q': 50, 'map': 0.06603235354497793, 'gm_map': 0.05126868034686285, 'bpref': 0.08373357630972357, 'Rprec': 0.0858115450641355, 'recip_rank': 0.9183333333333334, 'P@5': 0.8160000000000001, 'P@10': 0.7680000000000001, 'P@15': 0.7266666666666667, 'P@20': 0.7050000000000002, 'P@30': 0.664, 'P@100': 0.3618, 'P@200': 0.1809, 'P@500': 0.07236000000000001, 'P@1000': 0.036180000000000004, 'NDCG@5': 0.7558310986857175, 'NDCG@10': 0.7125345158714298, 'NDCG@15': 0.6781198651661505, 'NDCG@20': 0.657118300414957, 'NDCG@30': 0.6202108799800279, 'NDCG@100': 0.3904172391900226, 'NDCG@200': 0.24421554275576385, 'NDCG@500': 0.16421304918364274, 'NDCG@1000': 0.1550146349052302}


In [165]:
resultado_execucao['ndcg_10'] = round(100*results['NDCG@10'],2)

In [166]:
resultado_pipeline[nome_pipeline] = resultado_execucao
print(resultado_pipeline[nome_pipeline])

{'tempo_indexacao_segundo': [{'tipo': 'cpu', 'valor': 0.109634}], 'memoria_indice_byte_ram': 269772727, 'se_retrieval_usa_gpu': True, 'retrieval_tempo_medio_por_query': 1.722126, 'ndcg_10': 71.25}


In [167]:
avaliacao_pipeline_contexto[nome_pipeline] = retorna_calculo_contexto(resultado_execucao, parm_tipo_gpu='3090')
print(avaliacao_pipeline_contexto[nome_pipeline] )

para utilizacao_perfeita custo gpu dia: 5.999616
para utilizacao_precaria_100 custo gpu dia: 0.011958442944000001
{'utilizacao_perfeita': {'usd_query': 0.0001345363, 'usd_dia': 6.749761483605887, 'usd_gpu_dia': 5.999616, 'usd_mes': 202.49284450817663, 'usd_indexacao_tempo': 9.135801219999999e-07}, 'utilizacao_precaria_100': {'usd_query': 0.0076210393, 'usd_dia': 0.7621039265498879, 'usd_gpu_dia': 0.011958442944000001, 'usd_mes': 22.86311779649664, 'usd_indexacao_tempo': 9.135801219999999e-07}}


# Salvando dados

In [168]:
with open(PATH_AVALIACAO_CONTEXTO_PIPELINE, 'wb') as outputFile:
    pickle.dump(avaliacao_pipeline_contexto, outputFile, pickle.HIGHEST_PROTOCOL)

In [169]:
with open(PATH_RESULTADO_PIPELINE, 'wb') as outputFile:
    pickle.dump(resultado_pipeline, outputFile, pickle.HIGHEST_PROTOCOL)

# Carga dos resultados para novas avaliações

In [ ]:
with open(PATH_RESULTADO_PIPELINE, 'rb') as f:
    resultado_pipeline = pickle.load(f)

In [ ]:
with open(PATH_AVALIACAO_CONTEXTO_PIPELINE, 'rb') as f:
    avaliacao_pipeline_contexto = pickle.load(f)